# Pandas

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import xarray as xarr
import flammkuchen as fl

from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache
from inspyre import plotting as inplt
from sklearn.decomposition import PCA
from scipy.stats import pearsonr

In [2]:
import urllib, json

def get_unitn_hr_dataset():
    # This string contains the address at which we'll find the dataset:
    unitn_url = "https://dati.unitn.it/du/Person/en" 

    # Get page response:
    response = urllib.request.urlopen(unitn_url)

    # Parse a json from the page:
    json_data = json.loads(response.read())
    
    # Get actual data:
    data = json_data["value"]["data"]
    
    # Each person can have multiple positions. 
    # Here we go from a list of positions to positions in the dictionary, with a number in the 
    # key if there are multiple ones:
    
            
    return data # pd.DataFrame(data)

In [5]:
data = get_unitn_hr_dataset()

In [8]:
import json
with open("/Users/vigji/Desktop/unitn_people.json", "w") as f:
    json.dump(data, f)

In [4]:
url = "http://www.statweb.provincia.tn.it/indicatoristrutturalisubpro/exp.aspx?idind=648&info=d&fmt=json"
response = urllib.request.urlopen(url)

URLError: <urlopen error [Errno 60] Operation timed out>

In [ ]:
for d in data:
    positions = d["position"]#d.pop("position")
    titles = [p["role"] for p in positions]
    
    if "Staff" in titles and len(titles) > 1:
        print(titles)

In [85]:
[(d) for d in data if d["surname"]=="Adiletta"]

[{'identifier': 'f5193323aaaceff8c1ec459f7db2a823',
  'name': 'Alice',
  'surname': 'Adiletta',
  'phone': [],
  'position': [{'role': 'PhD student',
    'unitName': 'Doctoral programme - Cognitive and Brain Sciences',
    'unitId': 'STO0000432'}]}]

In [91]:
for d in data:
    positions = d["position"]#d.pop("position")
    titles = [p["role"] for p in positions]
    d["Research fellowship holder"] = "Research fellowship holder" in titles or "Research grant holder" in titles
    if len(titles) > 1:       
        for k in "PhD student", "Full professor", "Associate professor", "Teaching fellow", "Teaching assistant":
            if k in titles:
                valid_units = [p["unitName"] for p in positions if p["role"] == k]
                role = k
                break
    else:
        role = positions[0]["role"]
        valid_units = [positions[0]["unitId"]]
                
    d["role"] = role
    for i, unit in enumerate(valid_units):
        lab = f"_{i}" if i > 0 else ""
        d[f"unit{}"] = 

In [93]:
positions

[{'role': 'Teaching fellow',
  'unitName': 'Faculty of Law',
  'unitId': 'STO0008624'}]

In [60]:
df.groupby("role_2")["identifier"].count()

role_2
Adjunct professor                            2
Assistant professor                          4
Associate professor                         29
Chief digital transformation officer         1
Coordinator                                  2
Full professor                              52
Guest                                        4
Head                                         7
Member                                       2
PhD student                                  7
President                                    1
Rector's Delegate for musical activities     1
Rector's delegate for Disability Support     1
Research collaborator                        2
Research fellowship holder                  11
Staff                                       18
Students' representative                     3
Teaching assistant                           5
Teaching fellow                              3
Name: identifier, dtype: int64

In [46]:
# Each person can have multiple positions. 
# Here we go from a list of positions to positions in the dictionary, with a number in the 
# key if there are multiple ones:
for d in data:
    positions = d.pop("position")
    for i, position in enumerate(positions):
        i_lab = "" if i == 0 else f"_{i}"  # number in column name only for positions after first one
        d[f"role{i_lab}"] = position["role"]
        d[f"unit_name{i_lab}"] = position["unitName"]
        d[f"unit_id{i_lab}"] = position["unitId"]

In [48]:
pd.DataFrame(data)

,identifier,name,surname,phone,role,unit_name,unit_id,role_1,unit_name_1,unit_id_1,...,unit_id_6,role_7,unit_name_7,unit_id_7,role_8,unit_name_8,unit_id_8,role_9,unit_name_9,unit_id_9
0,3d22af80edc6910656c3e49f5e8213b8,Maurizio,Ferrari,[],Guest,Department of Physics,STO0008627,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1547a72aafb2b91e063ef3d978537a7a,Anna,Scalfi,[],Guest,Department of Sociology and social research,STO0008630,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9c0c137493b2ec94df0ad18f8c79fd40,Bruno,Callegher,[],Teaching fellow,Department of Humanities,STO0008632,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3f561286a93650282713dc900e080f98,Moreno,Ferrarese,[],Teaching fellow,Department of Sociology and social research,STO0008630,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,b69872e46afd8f1173b79b8f6b6765df,Paolo,Barbieri,"[0461 281326, 0461 281437]",Full professor,Doctoral School of Social Sciences,STO0000371,Full professor,Department of Sociology and social research,STO0008630,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4225,f8f8b51a648627dc1e1c615da692a941,Massimo,Bosetti,[],PhD student,PhD programme (Ministerial Decree 45/2013) - I...,STO0013634,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4226,5534a212b8cd842a1aa38f5c8e3c5bad,Federica,Arrigoni,[],Guest,Department of Information Engineering and Comp...,STO0008629,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4227,80fc8cd6b6a9fd978fe0c849ed600d1e,Mirella,Collini,[0461 281634],Head,"Management of national, EU and international r...",STO0013561,Head,Research Management Division,STO0013559,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4228,7c77a4a08257ddfbb2670eb207849406,Lorena,Leonardelli,[],Teaching assistant,"Department of Cellular, Computational and Inte...",STO0008662,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
